<a href="https://colab.research.google.com/github/arpitJaiswal550/food-delivery-time-prediction-hackthon/blob/main/food_delivery_time_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this hackthon I got 17th rank out of 1653 participants with 82.5% accuracy score.
And the Accuracy of 1st rank holder was 84.9%...

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook as tqdm
pd.set_option("max_columns", 1000)

In [2]:
# Mount the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train = pd.read_excel('/content/drive/MyDrive/Food_delivery_time_pred_dataset/Data_Train.xlsx')
print(train.shape)
train.head()

(11094, 9)


,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time
0,ID_6321,"FTI College, Law College Road, Pune","Fast Food, Rolls, Burger, Salad, Wraps",₹200,₹50,3.5,12,4,30 minutes
1,ID_2882,"Sector 3, Marathalli","Ice Cream, Desserts",₹100,₹50,3.5,11,4,30 minutes
2,ID_1595,Mumbai Central,"Italian, Street Food, Fast Food",₹150,₹50,3.6,99,30,65 minutes
3,ID_5929,"Sector 1, Noida","Mughlai, North Indian, Chinese",₹250,₹99,3.7,176,95,30 minutes
4,ID_6123,"Rmz Centennial, I Gate, Whitefield","Cafe, Beverages",₹200,₹99,3.2,521,235,65 minutes


In [4]:
test = pd.read_excel('/content/drive/MyDrive/Food_delivery_time_pred_dataset/Data_Test.xlsx')
print(test.shape)
test.head()

(2774, 8)


,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews
0,ID_2842,"Mico Layout, Stage 2, BTM Layout,Bangalore","North Indian, Chinese, Assamese",₹350,₹50,4.2,361,225
1,ID_730,"Mico Layout, Stage 2, BTM Layout,Bangalore","Biryani, Kebab",₹100,₹50,NEW,-,-
2,ID_4620,"Sector 1, Noida",Fast Food,₹100,₹50,3.6,36,16
3,ID_5470,"Babarpur, New Delhi, Delhi","Mithai, North Indian, Chinese, Fast Food, Sout...",₹200,₹50,3.6,66,33
4,ID_3249,"Sector 1, Noida","Chinese, Fast Food",₹150,₹50,2.9,38,14


In [5]:
submit = pd.read_excel('/content/drive/MyDrive/Food_delivery_time_pred_dataset/Sample_Submission.xlsx')
submit.head()

,Delivery_Time
0,120 minutes
1,20 minutes
2,20 minutes
3,65 minutes
4,45 minutes


In [6]:
train['Delivery_Time'].value_counts()

30 minutes     7406
45 minutes     2665
65 minutes      923
120 minutes      62
20 minutes       20
80 minutes       14
10 minutes        4
Name: Delivery_Time, dtype: int64

In [8]:
target = "Delivery_Time"
train_target = train[target]
train.drop([target], axis=1, inplace=True)

In [9]:
# Converting the classes to integer values. 
# Since this is a multi class classfication problem. The class mapping will be useful when ensembling various models.
train_target = train_target.apply(lambda x: x.split()[0]).astype(int)

class_map = {}
class_map_rev = {}
for a,b in enumerate(sorted(train_target.unique())):
    class_map[b] = a
    class_map_rev[a] = b
print("class mapping {}".format(class_map))
print("class mapping reverse {}".format(class_map_rev))

train_target = train_target.map(class_map)

class mapping {10: 0, 20: 1, 30: 2, 45: 3, 65: 4, 80: 5, 120: 6}
class mapping reverse {0: 10, 1: 20, 2: 30, 3: 45, 4: 65, 5: 80, 6: 120}


In [10]:
alldata = pd.concat([train, test], axis=0, sort=False, ignore_index=True)


def determine_unique_cuisines():
    cuisines_list = [val for val in alldata['Cuisines'].str.split(",")]
    cuisines_list = [",".join([v.strip() for v in val]) for val in cuisines_list]
    unique_cuisines = set(",".join(cuisines_list).split(","))
    print("total unique cuisines {}".format(len(unique_cuisines)))
    print("unique cuisines are {}".format(unique_cuisines))

    return unique_cuisines

unique_cuisines =  determine_unique_cuisines()

total unique cuisines 101
unique cuisines are {'Assamese', 'Healthy Food', 'Mithai', 'Seafood', 'Finger Food', 'Iranian', 'Modern Indian', 'Tex-Mex', 'Poké', 'Italian', 'Biryani', 'Coffee', 'Awadhi', 'Greek', 'Momos', 'Asian', 'Arabian', 'Vietnamese', 'Bakery', 'Steak', 'Rolls', 'Burmese', 'South Indian', 'BBQ', 'Roast Chicken', 'Lebanese', 'Belgian', 'Tea', 'Chinese', 'Indian', 'Maharashtrian', 'North Indian', 'Afghan', 'Hot dogs', 'Wraps', 'American', 'Chettinad', 'Tibetan', 'Mexican', 'Middle Eastern', 'Portuguese', 'Bohri', 'Sri Lankan', 'South American', 'Nepalese', 'African', 'Cafe', 'Brazilian', 'Mangalorean', 'Bubble Tea', 'Continental', 'Malaysian', 'Sushi', 'Gujarati', 'Parsi', 'Ice Cream', 'Turkish', 'Hyderabadi', 'Street Food', 'Bengali', 'German', 'Tamil', 'Malwani', 'Raw Meats', 'Juices', 'Bangladeshi', 'Pizza', 'Korean', 'French', 'Sandwich', 'Odia', 'Paan', 'Konkan', 'North Eastern', 'Spanish', 'Rajasthani', 'Goan', 'Salad', 'Desserts', 'Kerala', 'Mughlai', 'Cantonese',

In [11]:
cuisine_vals_list = []
for i, row in tqdm(alldata.iterrows()):
    location_dict = {k:0 for k in unique_cuisines}
    for k in row['Cuisines'].split(","):
        location_dict[k.strip()] = 1
    cuisine_vals_list.append(location_dict)
    
alldata = alldata.drop("Cuisines", axis=1)
alldata = pd.concat((alldata, pd.DataFrame(cuisine_vals_list).fillna(0).astype(np.int8)), axis=1)
alldata.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


,Restaurant,Location,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Assamese,Healthy Food,Mithai,Seafood,Finger Food,Iranian,Modern Indian,Tex-Mex,Poké,Italian,Biryani,Coffee,Awadhi,Greek,Momos,Asian,Arabian,Vietnamese,Bakery,Steak,Rolls,Burmese,South Indian,BBQ,Roast Chicken,Lebanese,Belgian,Tea,Chinese,Indian,Maharashtrian,North Indian,Afghan,Hot dogs,Wraps,American,Chettinad,Tibetan,Mexican,Middle Eastern,Portuguese,Bohri,Sri Lankan,South American,Nepalese,African,Cafe,Brazilian,Mangalorean,Bubble Tea,Continental,Malaysian,Sushi,Gujarati,Parsi,Ice Cream,Turkish,Hyderabadi,Street Food,Bengali,German,Tamil,Malwani,Raw Meats,Juices,Bangladeshi,Pizza,Korean,French,Sandwich,Odia,Paan,Konkan,North Eastern,Spanish,Rajasthani,Goan,Salad,Desserts,Kerala,Mughlai,Cantonese,Mishti,Kebab,Fast Food,European,Israeli,Lucknowi,Burger,Thai,Bihari,Mediterranean,Kashmiri,Bar Food,Charcoal Chicken,Naga,Beverages,Japanese,Andhra,Indonesian,Frozen Yogurt
0,ID_6321,"FTI College, Law College Road, Pune",₹200,₹50,3.5,12,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,ID_2882,"Sector 3, Marathalli",₹100,₹50,3.5,11,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,ID_1595,Mumbai Central,₹150,₹50,3.6,99,30,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,ID_5929,"Sector 1, Noida",₹250,₹99,3.7,176,95,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,ID_6123,"Rmz Centennial, I Gate, Whitefield",₹200,₹99,3.2,521,235,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [12]:
city = ['Pune', 'Bangalore', 'Mumbai', 'Noida', 'Bangalore', 'Delhi', 'Pune', 'Delhi', 'Kolkata',
        'Mumbai', 'Pune', 'Kolkata', 'Noida', 'Noida', 'Bangalore', 'Gurgaon', 'Kolkata',
        'Mumbai', 'Pune', 'Hyderabad', 'Bangalore', 'Kolkata', 'Delhi', 'Hyderabad',
        'Gurgaon', 'Delhi', 'Mumbai', 'Delhi', 'Pune', 'Hyderabad', 'Kolkata', 'Hyderabad', 'Hyderabad', 'Hyderabad', 'Bangalore']

location = alldata['Location'].unique().tolist()

city_mapping = dict(zip(location, city))
print("{:<15}{}".format('City', 'Location'))
print("".join("-"*50))
for key, val in city_mapping.items():
    print("{:<15}{}".format(val, key))

alldata['City'] = alldata['Location'].map(city_mapping)
alldata = alldata.drop("Location", axis=1)

City           Location
--------------------------------------------------
Pune           FTI College, Law College Road, Pune
Bangalore      Sector 3, Marathalli
Mumbai         Mumbai Central
Noida          Sector 1, Noida
Bangalore      Rmz Centennial, I Gate, Whitefield
Delhi          Delhi University-GTB Nagar
Pune           Yerawada, Pune, Maharashtra
Delhi          Delhi Administration Flats, Timarpur
Kolkata        Moulali, Kolkata
Mumbai         Dockyard Road, Mumbai CST Area
Pune           Pune University
Kolkata        Gora Bazar, Rajbari, North Dumdum, Kolkata
Noida          D-Block, Sector 63, Noida
Noida          Sector 14, Noida
Bangalore      Mico Layout, Stage 2, BTM Layout,Bangalore
Gurgaon        Laxman Vihar Industrial Area, Sector 3A, Gurgoan
Kolkata        Tiretti, Kolkata
Mumbai         Sandhurst Road, Mumbai CST Area
Pune           MG Road, Pune
Hyderabad      Hyderabad Public School, Begumpet
Bangalore      Majestic
Kolkata        Chandni Chowk, Kolkata
Delhi    

In [13]:
# Numeric feature cleaning
for col in ['Rating', 'Votes', 'Reviews']:
    alldata[col] = pd.to_numeric(alldata[col], errors='coerce').fillna(-99)
    
alldata['Average_Cost'] = pd.to_numeric(alldata['Average_Cost'].str[1:].str.replace(',',''), errors='coerce')
alldata['Average_Cost'] = alldata['Average_Cost'].fillna(-99).astype(int)

alldata['Minimum_Order'] = alldata['Minimum_Order'].str[1:].astype(int)

In [14]:
## Other usefull derived features
alldata['Minimum_Order_Zero'] = np.where(alldata['Minimum_Order'] == 0, 1, 0)

alldata['Reviews_by_Votes'] = alldata['Reviews'] / alldata['Votes']
alldata['Minimum_Order_to_Cost'] = alldata['Minimum_Order'] / alldata['Average_Cost']
alldata["Location_num_Res"] = alldata["City"].map(alldata.groupby("City").Restaurant.nunique())
alldata["Restaurant_branch_count"] = alldata["Restaurant"].map(alldata["Restaurant"].value_counts())

alldata.columns.tolist()

['Restaurant',
 'Average_Cost',
 'Minimum_Order',
 'Rating',
 'Votes',
 'Reviews',
 'Assamese',
 'Healthy Food',
 'Mithai',
 'Seafood',
 'Finger Food',
 'Iranian',
 'Modern Indian',
 'Tex-Mex',
 'Poké',
 'Italian',
 'Biryani',
 'Coffee',
 'Awadhi',
 'Greek',
 'Momos',
 'Asian',
 'Arabian',
 'Vietnamese',
 'Bakery',
 'Steak',
 'Rolls',
 'Burmese',
 'South Indian',
 'BBQ',
 'Roast Chicken',
 'Lebanese',
 'Belgian',
 'Tea',
 'Chinese',
 'Indian',
 'Maharashtrian',
 'North Indian',
 'Afghan',
 'Hot dogs',
 'Wraps',
 'American',
 'Chettinad',
 'Tibetan',
 'Mexican',
 'Middle Eastern',
 'Portuguese',
 'Bohri',
 'Sri Lankan',
 'South American',
 'Nepalese',
 'African',
 'Cafe',
 'Brazilian',
 'Mangalorean',
 'Bubble Tea',
 'Continental',
 'Malaysian',
 'Sushi',
 'Gujarati',
 'Parsi',
 'Ice Cream',
 'Turkish',
 'Hyderabadi',
 'Street Food',
 'Bengali',
 'German',
 'Tamil',
 'Malwani',
 'Raw Meats',
 'Juices',
 'Bangladeshi',
 'Pizza',
 'Korean',
 'French',
 'Sandwich',
 'Odia',
 'Paan',
 'Konk

In [15]:
num_cols = ['Votes', 'Reviews', 'Rating', 'Average_Cost', 'Minimum_Order', 
            'Restaurant_branch_count', 'Location_num_Res', 'Reviews_by_Votes', 'Minimum_Order_to_Cost']
cat_cols = [col for col in alldata.columns if col not in num_cols]
features = pd.get_dummies(alldata.drop(num_cols, axis=1), columns=cat_cols, sparse=True)

In [16]:
features = features.sparse.to_coo()

In [17]:
import scipy
num_features=scipy.sparse.coo_matrix(alldata[num_cols].values)

In [18]:
features=scipy.sparse.hstack([features, num_features]).tocsr()

In [19]:
train_ohe = features[:train.shape[0], :]
test_ohe = features[train.shape[0]:, :]

print(train_ohe.shape)
print(test_ohe.shape)

(11094, 8882)
(2774, 8882)


In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_ohe, train_target, test_size=0.20, random_state=314, stratify=train_target)

In [21]:

import lightgbm as lgb

lgb_fit_params={"early_stopping_rounds":50, 
            "eval_metric" : 'multi_logloss', 
            "eval_set" : [(X_test,y_test)],
            'eval_names': ['valid'],
            'verbose':100
           }

lgb_params = {'boosting_type': 'gbdt',
 'objective': 'multiclass',
 'metric': 'multi_logloss',
 'verbose': 0,
 'bagging_fraction': 0.8,
 'bagging_freq': 1,
 'num_class': 7,
 'feature_fraction': 0.8,
 'lambda_l1': 0.01,
 'lambda_l2': 0.01,
 'learning_rate': 0.1,
 'max_bin': 255,
 'max_depth': -1,
 'min_data_in_bin': 1,
 'min_data_in_leaf': 1,
 'num_leaves': 31}
lgb_params

{'bagging_fraction': 0.8,
 'bagging_freq': 1,
 'boosting_type': 'gbdt',
 'feature_fraction': 0.8,
 'lambda_l1': 0.01,
 'lambda_l2': 0.01,
 'learning_rate': 0.1,
 'max_bin': 255,
 'max_depth': -1,
 'metric': 'multi_logloss',
 'min_data_in_bin': 1,
 'min_data_in_leaf': 1,
 'num_class': 7,
 'num_leaves': 31,
 'objective': 'multiclass',
 'verbose': 0}

In [22]:
clf_lgb = lgb.LGBMClassifier(n_estimators=10000, **lgb_params, random_state=123456789, n_jobs=-1)
clf_lgb.fit(X_train, y_train, **lgb_fit_params)
clf_lgb.best_iteration_

Training until validation scores don't improve for 50 rounds.
[100]	valid's multi_logloss: 0.584989
[200]	valid's multi_logloss: 0.563424
[300]	valid's multi_logloss: 0.553254
Early stopping, best iteration is:
[338]	valid's multi_logloss: 0.552583


338

In [23]:
## model training on complete data
clf_lgb_fulldata = lgb.LGBMClassifier(n_estimators=int(clf_lgb.best_iteration_*1.2), **lgb_params)
clf_lgb_fulldata.fit(train_ohe, train_target)

LGBMClassifier(bagging_fraction=0.8, bagging_freq=1, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.8,
               importance_type='split', lambda_l1=0.01, lambda_l2=0.01,
               learning_rate=0.1, max_bin=255, max_depth=-1,
               metric='multi_logloss', min_child_samples=20,
               min_child_weight=0.001, min_data_in_bin=1, min_data_in_leaf=1,
               min_split_gain=0.0, n_estimators=405, n_jobs=-1, num_class=7,
               num_leaves=31, objective='multiclass', random_state=None,
               reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0, ...)

In [24]:
%%time
from sklearn.ensemble import RandomForestClassifier
clf_rf_fulldata=RandomForestClassifier(n_estimators=2000, max_features=0.1)
clf_rf_fulldata.fit(train_ohe, train_target)

CPU times: user 15min 16s, sys: 1.9 s, total: 15min 18s
Wall time: 15min 15s


In [27]:
## Ensembling lightgbm and RF models
predictions = np.mean((clf_lgb_fulldata.predict_proba(test_ohe), 
                       clf_rf_fulldata.predict_proba(test_ohe)), axis=0)
predictions = np.argmax(predictions, axis=1)

In [29]:
filename = 'submission_mean_ens_model1.xlsx'

# Make submission
submission = pd.DataFrame({target: pd.Series(predictions).map(class_map_rev).apply(lambda x: str(x)+" minutes")})
submission.to_excel(filename, index=False)
submission.head()

,Delivery_Time
0,30 minutes
1,30 minutes
2,30 minutes
3,30 minutes
4,30 minutes
